In [11]:
import geopandas as gpd
import pandas as pd 
import geopy
import numpy as np
import os
import re

In [12]:
crime = pd.read_csv('../input/Crime_Data_from_2010_to_2019.csv')
covs = pd.read_excel('../input/Affordable_Housing_Covenents.xlsx')
blocks = gpd.read_file('../input/LA_Blocks/tl_2010_06037_tabblock10.shp') # GPD has existing projection
population_cityofla = gpd.read_file('../input/Census_Blocks_2010_Population/Census_Blocks_2010_Population.shp')

In [13]:
city_blocks = population_cityofla[['GEOID10']]

In [14]:
# These are the Census Blocks in Los Angeles!

blocks = pd.merge(city_blocks, blocks, on= 'GEOID10', how='inner')
blocks = gpd.GeoDataFrame(blocks.drop(columns = 'geometry'), geometry=blocks['geometry'], crs= 4269)

In [15]:
print(f'The crime dataset has {crime.shape[0]} observations.')
print(f'The affordable housing dataset has {covs.shape[0]} observations.')
print(f'The affordable housing dataset has {blocks.shape[0]} observations. Meaning, there are {blocks.shape[0]} U.S. Census Blocks.')


The crime dataset has 2060948 observations.
The affordable housing dataset has 1302 observations.
The affordable housing dataset has 30691 observations. Meaning, there are 30691 U.S. Census Blocks.


In [16]:
# Transforming the Block CRS:

blocks = blocks.to_crs("EPSG:4326")

In [17]:
# Turning our PDs to GPDs:

crime = gpd.GeoDataFrame(crime, geometry= gpd.points_from_xy(crime.LON, crime.LAT), crs=4326)
covs = gpd.GeoDataFrame(covs, geometry= gpd.points_from_xy(covs.Lng, covs.Lat), crs=4326)

In [18]:
# Saving the covenents for mapping:

covs.to_file('../intermediate/affordable_covenents/affordable_covenents.shp')

<ipython-input-18-52b10d095660>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  covs.to_file('../intermediate/affordable_covenents/affordable_covenents.shp')


In [19]:
# Dropping crime data that are not geolocated:

crime_notnull = crime[~(crime["LAT"] == 0)]
print(crime_notnull.shape[0]/crime.shape[0])

crime = crime_notnull

0.9990538334785739


In [20]:
crime.shape[0] - crime_notnull.shape[0]

0

We lose less than .1% of our data when we drop incidents of crime that do not have coordinates.

#### __Joining Crime Data with Block Shapefile:__

In [21]:
crime['date'] = pd.to_datetime(crime['DATE OCC'])
crime['year'] = pd.DatetimeIndex(crime['date']).year
crime['month'] = pd.DatetimeIndex(crime['date']).month
crime['year'] = crime['year'].astype(str)

In [22]:
pd.set_option("display.max_columns", None)
crime['unique_id'] = crime.reset_index().index
pd.pivot_table(crime[['unique_id', 'year', 'month']], index='year', columns='month', aggfunc= 'count')

unique_id                                                          \
month        1      2      3      4      5      6      7      8      9    
year                                                                      
2010      19469  16021  18127  17771  17716  17672  17851  17294  16628   
2011      18097  14677  16939  16433  16938  17014  17268  17001  16241   
2012       5963   5543   5717   5728   6163   5384   5233   5538   5248   
2013      16765  14061  16216  15719  16595  16097  16630  17427  16220   
2014      16181  13681  15601  15408  16699  16360  17222  17044  16743   
2015      18386  16006  18005  16944  17939  17546  18685  18990  18276   
2016       6816   6403   6571   6754   6846   6574   6990   6840   5889   
2017      19671  17110  19411  18752  19452  19043  20163  19720  19420   
2018      19401  17141  19027  19132  19941  19271  19833  19723  18445   
2019      18454  16258  18799  17923  18642  18335  19074  18911  17990   
2020      10809   9884   9243   9523  10599  10993  10715  10584   9727   
2021       8519   7797   9440   9668  10414  10400  11134  11090  10912   

                            
month     10     11     12  
year                        
2010   17616  16007  17029  
2011   17030  16417  16681  
2012    5335   4696   4362  
2013   16236  15337  15339  
2014   17301  15947  17170  
2015   19203  17377  18207  
2016    6177   5547   5305  
2017   20400  19011  19023  
2018   19521  18650  18923  
2019   18198  17217  18001  
2020    8696   7827   7123  
2021   11224   9558     11

In [12]:
crime.head(1)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,geometry,date,year
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695,POINT (-118.26950 33.98250),2010-02-20,2010


In [13]:
blocks.head(1)

,GEOID10,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,060372626043000,06,037,262604,3000,Block 3000,G5040,U,51445,U,S,389838,0,+34.0473001,-118.5567635,"POLYGON ((-118.55912 34.04711, -118.55965 34.0..."


In [14]:
points = crime[['year', 'geometry']]
polygons = blocks[['GEOID10', 'geometry']]

temp = gpd.sjoin(points, polygons, how='inner', op='intersects')

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [15]:
temp["const"] = 1

In [16]:
only_crimes_by_block = pd.pivot_table(temp, values='const', index=['GEOID10'],
                    columns=['year'], aggfunc=np.sum)

In [17]:
only_crimes_by_block = only_crimes_by_block.fillna(0)

In [18]:
only_crimes_by_block['Total'] = only_crimes_by_block.sum(axis=1)

In [19]:
only_crimes_by_block.shape

(24166, 13)

In [20]:
print(f'There are {30691 - only_crimes_by_block.shape[0]} blocks that never experienced a crime incident.')


There are 6525 blocks that never experienced a crime incident.


In [21]:
crimes_by_block = pd.merge(blocks, only_crimes_by_block, on='GEOID10', how="left")

In [22]:
crimes_by_block = crimes_by_block.fillna(0)
crimes_by_block.shape # Same number of observations as the original Block Data.

(30691, 29)

In [23]:
crimes_by_block[['2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019']].sum().sum()

1820212.0

In [24]:
# Dropping 2020, 2021 years

crimes_by_block = crimes_by_block[['geometry', 'GEOID10', '2010', '2011', 
                                   '2012', '2013', '2014', '2015', 
                                   '2016', '2017', '2018', '2019']] 

#### __Joining Covenent Data with Block Shapefile:__

In [25]:
covs.head(5)

,Address,Zip Code,Machine Readable Address,Year of Covenant,Year of Year of Covenant,Council District,Covenant,Year of Covenant.1,% Affordable,Affordable %,Affordable Units,Filter Covenant Year,Lat,Lng,Total Units,geometry
0,"8747 N.Sepulveda Blvd, North Hills",91301,"8747 N.Sepulveda Blvd, North Hills",2010,2010,6,2010,2010,0.105263,10.526316,4,2010,34.229631,-118.468193,38,POINT (-118.46819 34.22963)
1,"14349-14353 W. Friar St, Van Nuys",91401,"14349 W. Friar St, Van Nuys",2010,2010,6,2010,2010,0.111111,11.111111,2,2010,34.185966,-118.446006,18,POINT (-118.44601 34.18597)
2,"1813-1819 Prosser, Los Angeles",90027,"1813 Prosser, Los Angeles",2010,2010,5,2010,2010,0.095238,9.523810,2,2010,34.052958,-118.429181,21,POINT (-118.42918 34.05296)
3,"510 Landfair Ave, Los Angeles",90024,"510 Landfair Ave, Los Angeles",2010,2010,5,2010,2010,0.055556,5.555556,2,2010,34.068512,-118.450467,36,POINT (-118.45047 34.06851)
4,"11901 Goshen Ave, Los Angeles",90049,"11901 Goshen Ave, Los Angeles",2010,2010,11,2010,2010,0.004545,0.454545,1,2010,34.047861,-118.465273,220,POINT (-118.46527 34.04786)


In [26]:
covs['Year of Covenant'] = covs['Year of Covenant'].astype(str)

In [27]:
points = covs[['Year of Covenant', 'geometry']]
polygons = blocks[['GEOID10', 'geometry']]

temp_2 = gpd.sjoin(points, polygons, how='inner', op='intersects')

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [28]:
first_covs = temp_2.sort_values(by=['Year of Covenant']).drop_duplicates(subset=['GEOID10']).rename(columns={'Year of Covenant': 'yfirst_cov'})
first_covs = first_covs[['yfirst_cov', 'GEOID10']]

In [29]:
temp_2['const'] = 1

In [30]:
covs_by_block_NOAGG = pd.pivot_table(temp_2, values='const', index=['GEOID10'],
                    columns=['Year of Covenant'], aggfunc=np.sum)
only_covs_by_block = covs_by_block_NOAGG.fillna(0)

In [31]:
covs_by_block_NOAGG.sum().sum() # The original table had 1302 observations.

1283.0

In [32]:
only_covs_by_block = only_covs_by_block.cumsum(axis = 1)

In [33]:
## ASSUMPTION:
only_covs_by_block = only_covs_by_block.mask(only_covs_by_block > 1,1)

In [34]:
only_covs_by_block.shape[0]

1041

In [35]:
print(f'There are {30691 - only_covs_by_block.shape[0]} blocks that never received a affordable housing covenent.')


There are 29650 blocks that never received a affordable housing covenent.


In [36]:
covs_by_block = pd.merge(blocks, only_covs_by_block, on='GEOID10', how= "left")

In [37]:
covs_by_block = covs_by_block[['geometry', 'GEOID10', '2010', '2011', 
                                   '2012', '2013', '2014', '2015', '2016', 
                               '2017', '2018', '2019', '2020', '2021']].fillna(0)

In [38]:
covs_by_block

,geometry,GEOID10,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,"POLYGON ((-118.55912 34.04711, -118.55965 34.0...",060372626043000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"POLYGON ((-118.55799 34.04871, -118.55801 34.0...",060372626043001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"POLYGON ((-118.55912 34.04711, -118.55973 34.0...",060379800191020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"POLYGON ((-118.56866 34.07184, -118.56898 34.0...",060379800191013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"POLYGON ((-118.56769 34.07935, -118.56771 34.0...",060372626011011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30686,"POLYGON ((-118.15913 34.09860, -118.15913 34.0...",060374807033012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30687,"POLYGON ((-118.15671 34.09860, -118.15671 34.0...",060372011203001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30688,"POLYGON ((-118.15795 34.09867, -118.15678 34.0...",060374807033011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30689,"POLYGON ((-118.15536 34.09858, -118.15529 34.0...",060372011203000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [39]:
# FIND YEAR FOR WHICH THE BLOCK RECEIVED ItS FIRST COVENENt

#### __Constructing the dataset for this paper:__

In [40]:
# How many blocks that received an affordable housing covenent have crime data?

only_data = pd.merge(only_covs_by_block, only_crimes_by_block, 
                     on= 'GEOID10', suffixes=('_covs', '_crimes'))
print(f'There are {only_data.shape[0]} blocks that received an affordable housing covenent and has existing crime data.')



There are 954 blocks that received an affordable housing covenent and has existing crime data.


In [41]:
dataset = pd.merge(covs_by_block, 
                   crimes_by_block, on= 'GEOID10', suffixes=('_covs', '_crimes')).rename(columns={"2020": "2020_covs", "2021": "2021_covs"})

Each set is one block, when you go to run your regression is to make is it long, each obersvation is going to be a year and a block
- use reshape in STATA

In [42]:
# Each set is one block, when you go to run your regression is to make is it long, each obersvation is going to be a year and a block, wide have to make fummy
dataset


,geometry_covs,GEOID10,2010_covs,2011_covs,2012_covs,2013_covs,2014_covs,2015_covs,2016_covs,2017_covs,...,2010_crimes,2011_crimes,2012_crimes,2013_crimes,2014_crimes,2015_crimes,2016_crimes,2017_crimes,2018_crimes,2019_crimes
0,"POLYGON ((-118.55912 34.04711, -118.55965 34.0...",060372626043000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,20.0,9.0,30.0,18.0,25.0,14.0,18.0,26.0,16.0
1,"POLYGON ((-118.55799 34.04871, -118.55801 34.0...",060372626043001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,3.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2,"POLYGON ((-118.55912 34.04711, -118.55973 34.0...",060379800191020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,"POLYGON ((-118.56866 34.07184, -118.56898 34.0...",060379800191013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"POLYGON ((-118.56769 34.07935, -118.56771 34.0...",060372626011011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,3.0,0.0,0.0,1.0,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30686,"POLYGON ((-118.15913 34.09860, -118.15913 34.0...",060374807033012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30687,"POLYGON ((-118.15671 34.09860, -118.15671 34.0...",060372011203001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,3.0,0.0,2.0,0.0,1.0
30688,"POLYGON ((-118.15795 34.09867, -118.15678 34.0...",060374807033011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30689,"POLYGON ((-118.15536 34.09858, -118.15529 34.0...",060372011203000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,14.0,8.0,26.0,19.0,12.0,1.0,2.0,4.0,3.0,1.0


In [43]:
dataset = gpd.GeoDataFrame(dataset.drop(columns = ['geometry_covs', 
                                                  'geometry_crimes']), geometry=dataset['geometry_crimes'], crs= 4326)


In [44]:
dataset.columns

Index(['GEOID10', '2010_covs', '2011_covs', '2012_covs', '2013_covs',
       '2014_covs', '2015_covs', '2016_covs', '2017_covs', '2018_covs',
       '2019_covs', '2020_covs', '2021_covs', '2010_crimes', '2011_crimes',
       '2012_crimes', '2013_crimes', '2014_crimes', '2015_crimes',
       '2016_crimes', '2017_crimes', '2018_crimes', '2019_crimes', 'geometry'],
      dtype='object')

In [45]:
dataset['total_crimes'] = dataset['2010_crimes'] + dataset['2011_crimes'] + dataset['2012_crimes']
dataset['total_crimes'] = dataset['total_crimes'] + dataset['2013_crimes'] + dataset['2014_crimes']
dataset['total_crimes'] = dataset['total_crimes'] + dataset['2015_crimes'] + dataset['2016_crimes']
dataset['total_crimes'] = dataset['total_crimes'] + dataset['2017_crimes'] + dataset['2018_crimes']
dataset['total_crimes'] = dataset['total_crimes'] + dataset['2019_crimes'] 

In [46]:
dataset = dataset.to_crs(2229) # reprojecting for maps

In [47]:
# dataset.to_file('../intermediate/dataset/dataset.shp')

In [48]:
dataset_normalized = dataset.merge(population_cityofla, how= 'inner',  on= 'GEOID10')

In [49]:
dataset_normalized = dataset_normalized[~(dataset_normalized['POP'] == 0)] # dropping observations where block population equals 0.
# The code above could also be done in STATA.

dataset_normalized['2010_crimes'] = (dataset_normalized['2010_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2011_crimes'] = (dataset_normalized['2011_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2012_crimes'] = (dataset_normalized['2012_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2013_crimes'] = (dataset_normalized['2013_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2014_crimes'] = (dataset_normalized['2014_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2015_crimes'] = (dataset_normalized['2015_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2016_crimes'] = (dataset_normalized['2016_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2017_crimes'] = (dataset_normalized['2017_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2018_crimes'] = (dataset_normalized['2018_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized['2019_crimes'] = (dataset_normalized['2019_crimes'] / dataset_normalized['POP']) * 100

dataset_normalized = gpd.GeoDataFrame(dataset_normalized.drop(columns=['geometry_x', 'geometry_y']), 
                                      geometry= dataset_normalized['geometry_x'], crs= "EPSG:2229")

In [50]:
## Which blocks allow for multifamily development?
zoning = gpd.read_file('../input/Los Angeles_zoning.geojson')

In [51]:
zoning_SF = zoning[['zone_cat', 'geometry']]
zoning_SF = zoning_SF[zoning_SF.zone_cat == 'SF']
zoning_SF = zoning_SF.rename(columns= {'zone_cat': 'perm_SF'})
zoning_SF = zoning_SF.to_crs('EPSG:2229')

df = gpd.sjoin(dataset_normalized, zoning_SF, how='left', op='intersects')
df = df[['GEOID10', 'perm_SF']].drop_duplicates()
dataset_normalized = dataset_normalized.merge(df, how = 'left', on= 'GEOID10')
dataset_normalized['perm_SF'] = dataset_normalized['perm_SF'].replace({'SF': 1}).fillna(0)

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [52]:
zoning_MF = zoning[['zone_cat', 'geometry']]
zoning_MF = zoning_MF[zoning_MF.zone_cat == 'MF']
zoning_MF = zoning_MF.rename(columns= {'zone_cat': 'perm_MF'})
zoning_MF = zoning_MF.to_crs('EPSG:2229')

df = gpd.sjoin(dataset_normalized, zoning_MF, how='left', op='intersects')
df = df[['GEOID10', 'perm_MF']].drop_duplicates()
dataset_normalized = dataset_normalized.merge(df, how = 'left', on= 'GEOID10')
dataset_normalized['perm_MF'] = dataset_normalized['perm_MF'].replace({'MF': 1}).fillna(0)

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [53]:
zoning_MIX = zoning[['zone_cat', 'geometry']]
zoning_MIX = zoning_MIX[zoning_MIX.zone_cat == 'MIX']
zoning_MIX = zoning_MIX.rename(columns= {'zone_cat': 'perm_MIX'})
zoning_MIX = zoning_MIX.to_crs('EPSG:2229')

df = gpd.sjoin(dataset_normalized, zoning_MIX, how='left', op='intersects')
df = df[['GEOID10', 'perm_MIX']].drop_duplicates()
dataset_normalized = dataset_normalized.merge(df, how = 'left', on= 'GEOID10')
dataset_normalized['perm_MIX'] = dataset_normalized['perm_MIX'].replace({'MIX': 1}).fillna(0)

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [54]:
# dataset_normalized.to_file('../intermediate/dataset/dataset.shp')
# dataset_normalized.to_excel("../intermediate/dataset/dataset.xlsx", index= False, header= True)

In [55]:
# if the block was treated, when was it treated?
dataset_normalized = dataset_normalized.merge(first_covs, how = 'left', on= 'GEOID10')
dataset_normalized['yfirst_cov'] = dataset_normalized['yfirst_cov'].fillna('not_treated')

In [56]:
dataset_normalized[(dataset_normalized['yfirst_cov'].isin(['2020', '2021'])) & (dataset_normalized['POP'] <= 99)].shape

(44, 55)

In [57]:
dataset_normalized[(dataset_normalized['yfirst_cov'].isin(['2020', '2021']))].shape

(209, 55)

In [58]:
first_covs[first_covs['yfirst_cov'].isin(['2020', '2021'])]

,yfirst_cov,GEOID10
1111,2020,060372393302004
1201,2020,060372414002025
1108,2020,060372393301006
1193,2020,060372077101036
1107,2020,060372240202003
...,...,...
1208,2021,060372060101020
1207,2021,060371977001000
1206,2021,060372122042000
1204,2021,060372122021002


In [59]:
# what is a block's distance from nearest affordable housing covenent, for each year

In [60]:
for_nearest_dist = covs[['Year of Covenant', 'geometry']]
for_nearest_dist = for_nearest_dist.to_crs(2229)
blank = dataset_normalized[['GEOID10', 'geometry']]

In [61]:
nearest_2010 = for_nearest_dist[for_nearest_dist['Year of Covenant'] == '2010']
nearest_2011 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2011']
nearest_2012 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2012']
nearest_2013 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2013']
nearest_2014 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2014']
nearest_2015 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2015']
nearest_2016 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2016']
nearest_2017 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2017']
nearest_2018 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2018']
nearest_2019 = for_nearest_dist[for_nearest_dist['Year of Covenant'] <= '2019']

In [62]:
df2010 = gpd.sjoin_nearest(blank, nearest_2010, distance_col='d2010', how='left')[['GEOID10', 'd2010']]
df2011 = gpd.sjoin_nearest(blank, nearest_2011, distance_col='d2011', how='left')[['GEOID10', 'd2011']]
df2012 = gpd.sjoin_nearest(blank, nearest_2012, distance_col='d2012', how='left')[['GEOID10', 'd2012']]
df2013 = gpd.sjoin_nearest(blank, nearest_2013, distance_col='d2013', how='left')[['GEOID10', 'd2013']]
df2014 = gpd.sjoin_nearest(blank, nearest_2014, distance_col='d2014', how='left')[['GEOID10', 'd2014']]
df2015 = gpd.sjoin_nearest(blank, nearest_2015, distance_col='d2015', how='left')[['GEOID10', 'd2015']]
df2016 = gpd.sjoin_nearest(blank, nearest_2016, distance_col='d2016', how='left')[['GEOID10', 'd2016']]
df2017 = gpd.sjoin_nearest(blank, nearest_2017, distance_col='d2017', how='left')[['GEOID10', 'd2017']]
df2018 = gpd.sjoin_nearest(blank, nearest_2018, distance_col='d2018', how='left')[['GEOID10', 'd2018']]
df2019 = gpd.sjoin_nearest(blank, nearest_2019, distance_col='d2019', how='left')[['GEOID10', 'd2019']]


In [63]:
distances_df = blank.merge(df2010, how= 'left', on= 'GEOID10').merge(df2011, how= 'left', on= 'GEOID10').merge(
    df2012, how= 'left', on= 'GEOID10').merge(df2013, how= 'left', on= 'GEOID10').merge(
    df2014, how= 'left', on= 'GEOID10').merge(df2015, how= 'left', on= 'GEOID10').merge(
    df2016, how= 'left', on= 'GEOID10').merge(df2017, how= 'left', on= 'GEOID10').merge(df2018, how= 'left', on= 'GEOID10').merge(
    df2019, how= 'left', on= 'GEOID10')
distances_df = distances_df.drop_duplicates()
dataset_normalized = dataset_normalized.merge(distances_df, how= 'left', on= 'GEOID10')

In [64]:
# projects that are big and projects that are almost 100% affordable, a manager's unit is not offered below market
# I consider a project greater than 30 units to be a large project (find justification)

In [65]:
df_aff = covs[covs['% Affordable'] >= .95]
df_aff['pct95aff'] = 1
df_aff = df_aff[['geometry', 'pct95aff']].to_crs(2229)
df_aff = blank.sjoin(df_aff).drop_duplicates(subset=['GEOID10'])[['GEOID10', 'pct95aff']]
dataset_normalized = dataset_normalized.merge(df_aff, how= 'left', on= 'GEOID10')
dataset_normalized['pct95aff'] = dataset_normalized['pct95aff'].replace({np.nan: 0})

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [66]:
df_large = covs[covs['Affordable Units'] >= 30]
df_large['cont_large'] = 1
df_large = df_large[['geometry', 'cont_large']].to_crs(2229)
df_large = blank.sjoin(df_large).drop_duplicates(subset=['GEOID10'])[['GEOID10', 'cont_large']]
dataset_normalized = dataset_normalized.merge(df_large, how= 'left', on= 'GEOID10')
dataset_normalized['cont_large'] = dataset_normalized['cont_large'].replace({np.nan: 0})

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [67]:
df_small = covs[covs['Affordable Units'] <= 10]
df_small['cont_small'] = 1
df_small = df_small[['geometry', 'cont_small']].to_crs(2229)
df_small = blank.sjoin(df_small).drop_duplicates(subset=['GEOID10'])[['GEOID10', 'cont_small']]
dataset_normalized = dataset_normalized.merge(df_small, how= 'left', on= 'GEOID10')
dataset_normalized['cont_small'] = dataset_normalized['cont_small'].replace({np.nan: 0})

/Users/vivekadury/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [68]:
df = covs.to_crs(2229)
df = blank.sjoin(df).sort_values('Year of Covenant').drop_duplicates(subset=['GEOID10'])[['GEOID10', 'Affordable %', 'Affordable Units', 'Total Units']]
df = df.rename(columns={'Affordable %': 'Treated_Pct_Aff', 'Affordable Units': 'Treated_Aff_Units', 'Total Units': 'Treated_TOT_Units'})
dataset_normalized = dataset_normalized.merge(df, how= 'left', on= 'GEOID10')

In [69]:
dataset_normalized = gpd.GeoDataFrame(dataset_normalized.drop(columns=['geometry_x', 'geometry_y']), 
                                      geometry= dataset_normalized['geometry_x'], crs= "EPSG:2229")
dataset_normalized.to_file('../intermediate/dataset/dataset.shp')
dataset_normalized.to_excel("../intermediate/dataset/dataset.xlsx", index= False, header= True)

<ipython-input-69-e614a4eec14c>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  dataset_normalized.to_file('../intermediate/dataset/dataset.shp')
